In [116]:
#library for image processing
import cv2
import numpy as np
#import imutils
import pandas as pd
import cvlib as cv
from cvlib.object_detection import draw_bbox

#library for image processing
import serial
import time

## <span style="color:orange">*set up port*</span>

In [100]:
#setting up port for arduino
arduino = serial.Serial(port='COM3', baudrate=9600, timeout=.1)

## <span style="color:orange">*Send signal to arduino*</span>

In [117]:
def write_signal(i):
    # to drive motor 1
    if (i == 1):
        arduino.write(bytes(b'1'))
    # to drive motor 2
    elif (i == 2):
        arduino.write(bytes(b'2'))
    # to drive motor 3
    elif (i == 3):
        arduino.write(bytes(b'3'))
    # light red led up
    elif (i == 'r'):
        arduino.write(bytes(b'r'))
    # light green led up
    elif (i == 'g'):
        arduino.write(bytes(b'g'))
    # light yellow led up
    elif (i == 'y'):
        arduino.write(bytes(b'y'))
    #stop all motor
    elif (i == 'q'):
        arduino.write(bytes(b'q'))
    #stop led
    elif (i == 'm'):
        arduino.write(bytes(b'm'))
    
    time.sleep(0.05)
    data = arduino.readline()
    return data

## <span style="color:orange">*Image Preprocessing*</span>

In [105]:
def image_processing():
    # Create a VideoCapture object and read from input file
    # If the input is the camera, pass 0 instead of the video file name
    cap = cv2.VideoCapture(0)
    # Check if camera opened successfully
    if (cap.isOpened()== False): 
        print("Error opening video stream or file")
    # Read until video is completed
    time_delay = 10
    while cap.isOpened():
    
    #time delay(s)
        
    
    # Capture frame-by-frame
        ret, frame = cap.read()
      
        if frame is None:
            print('frame is none')
            break
    # Saving the frames with certain namec
        if ret == True:
        
            #build in object detection 
            box, label, count = cv.detect_common_objects(frame)
            output = draw_bbox(frame, box, label, count)
            
            #if found something
            if len(label) > 0:
                write_signal("y")
                
            #if trash > 1
            if len(label) > 1:
                print("case1")
                print(label)
                return(0)
            #if a trash is not a bottle
            elif (len(label) == 1 and label[0] != 'bottle'):
                print("case2")
                print(label)
                return (0)
            
            #if a trash is a bottle
            elif (len(label) == 1 and label[0] == 'bottle'):
                print("case3")
                print(label)
                return (1)

        print(label)
    
        #decreament
        time_delay -= 1
        time.sleep(1)
        print(time_delay)
        if not(time_delay):
            return -1


## <span style="color:orange">*Main program*</span>

In [ ]:
excel_file_path = './info_data.xlsx' #locate path to variable
df = pd.read_excel(excel_file_path) #load database from excel turn into df
ldf = df.to_numpy().tolist()#typecast df to list

for i in range (len(ldf)):
    ldf[i] = ldf[i][1:]#remove index
    ldf[i][1] = str(ldf[i][1])#typecase id from int->str

while True:
    write_signal('q') #starting by stop motor
    input_data = input("Scan your qrcode") #scan qr code 

    door_open = 0

    for i in range(len(ldf)):
        if input_data in ldf[i]: # if id is found
            print("The door is opening")
            print("Hello ", ldf[i][0])
            door_open = 1
            break
        elif (i == len(ldf)-1): # if id is not found
            print("Database doesn't have your info")
            add_name = str(input("Enter your name: "))
            ldf.append([add_name, input_data, 10]) # add data
            door_open = 1
    
    while True:
        print("========door open========")
        write_signal('m')
        write_signal('q')
        if (door_open):
            write_signal(1) #open main gate
            print("========image processing========")
            status = image_processing() #do the image processing
            if status == -1: # if no trash for 10 sec
                write_signal('m')
                break
            write_signal('q')
            time.sleep(5)
            if (status): # if PET
                ldf[i][2] += 0.5
                write_signal('m')
                write_signal('g')
                write_signal(2)
                time.sleep(5)

            elif not(status): #if not PET or trash > 1
                ldf[i][2] -= 1
                write_signal('m')
                write_signal('r')
                write_signal(3)
                time.sleep(5)
        print("score = ", ldf[i][2])
                
    #print(ldf[i])
    df = pd.DataFrame(np.array(ldf))
    #df.rename(columns={0: "Name", 1: "ID", 2: "Score"})
    df.to_excel(excel_file_path)